In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import root_scalar

In [ ]:
#Constants

R = 8.314
P = 1

A_but = 9.9614
B_but = 2664.0939
C_but = -104.881
A_h2O = 11.9647
B_h2O = 3984.9273
C_h2O = -39.734

g12_g22 = 1034.3
g21_g11 = 10098.50
aij = 0.4118

In [ ]:
def molfrac_vapor(T, x):
    x_but = x
    x_h2O = 1 - x

    tau_12 = g12_g22 / (R * T)
    tau_21 = g21_g11 / (R * T)

    G_12 = np.exp(-aij * tau_12)
    G_21 = np.exp(-aij * tau_21)

    gamma_but = np.exp(x_h2O**2 * (tau_21 * (G_21 / (x_but + x_h2O * G_21))**2 + (tau_12 * G_12) / (x_h2O + x_but * G_12)**2))
    gamma_h2O = np.exp(x_but**2 * (tau_12 * (G_12 / (x_h2O + x_but * G_12))**2 + (tau_21 * G_21) / (x_but + x_h2O * G_21)**2))
    
    Psat_but = np.exp(A_but - B_but / (T + C_but))
    Psat_h2O = np.exp(A_h2O - B_h2O / (T + C_h2O))
    
    y_but = gamma_but * x_but * Psat_but / P
    y_h2O = gamma_h2O * x_h2O * Psat_h2O / P
    
    sum_y = y_but + y_h2O
    
    return sum_y - 1  

In [ ]:
x_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
T_solutions = []

for i,x in enumerate(x_values):
    sol, = fsolve(molfrac_vapor, 373, x)
    T_solutions.append(sol)

print("Solutions for T:", T_solutions)
print(f"mean: {np.mean(T_solutions)}")

In [ ]:
T_min = 300  # Minimum temperature in K
T_max = 450  # Maximum temperature in K

x_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
T_solutions = []

for i, x in enumerate(x_values):
    result = root_scalar(molfrac_vapor, bracket=[T_min, T_max], args=x)
    print(f"result: {result}")
    
    if result.converged:
        T_solutions.append(result.root)
    else:
        T_solutions.append(None)
        print(f"No solution found for x = {x} within the range [{T_min}, {T_max}]")

print("Solutions for T:", T_solutions)